In [1]:
import nevergrad as ng

# Basic usage

We create a simple function with one minimum at $x = 0.5$.

In [6]:
def square(x, y=12):
    """
    Convex objective function
    """
    return sum((x - 1.5) ** 2) + abs(y)

Let's optimize (minimize) the function value with respect to the function `args`. 
We will search for an $x$ value that minimizes the objective function `square(x, y=12)`.

We can access the global optimizers classes at `ng.optimizers.`.

`OnePlusOne` is one of the supported optimizers in `nevergrad`.


- `instrumentation`: with an argument of type `int` defines the dimension of the input search space.

- `budget`: defines the allowed number of objective evaluations

The `x` variable can be a vector of dimension `x_dimension`.

In [13]:
x_dimension = 2
optimizer = ng.optimizers.OnePlusOne(instrumentation=x_dimension, budget=100)
recommendation = optimizer.minimize(square)
print(f"Recommendation arguments: {recommendation.args}")
print(f"Recommendation keyword arguments: {recommendation.kwargs}")

Recommendation arguments: (array([1.50004486, 1.49942011]),)
Recommendation keyword arguments: {}


From the official documentation:

In [28]:
help(optimizer.minimize)

Help on method minimize in module nevergrad.optimization.base:

minimize(objective_function:Callable[..., float], executor:Union[nevergrad.common.typetools.ExecutorLike, NoneType]=None, batch_mode:bool=False, verbosity:int=0) -> nevergrad.optimization.base.Candidate method of nevergrad.optimization.optimizerlib._OnePlusOne instance
    Optimization (minimization) procedure
    
    Parameters
    ----------
    objective_function: callable
        A callable to optimize (minimize)
    executor: Executor
        An executor object, with method `submit(callable, *args, **kwargs)` and returning a Future-like object
        with methods `done() -> bool` and `result() -> float`. The executor role is to dispatch the execution of
        the jobs locally/on a cluster/with multithreading depending on the implementation.
        Eg: `concurrent.futures.ThreadPoolExecutor`
    batch_mode: bool
        when num_workers = n > 1, whether jobs are executed by batch (n function evaluations are launch

# Instrumentation: dealing with generic input data

It is possible to define an input parameter space taylored for your needs.

Whether your objective uses numerical or categorical values, `nevergrad` can do that for you.

## Using `Scalar` variables

Initializing a `Scalar` variable with normal gaussian prior $\mathcal{N}\left(0, 1\right)$, and custom gaussian prior $\mathcal{N}\left(b, a\right)$:

In [41]:
scalar_variable = ng.var.Scalar()
print(f"scalar variable: {scalar_variable.name}")

affined_scalar_variable = ng.var.Scalar().affined(a=2, b=-1)
print(f"scalar variable with affined transformation: {affined_scalar_variable.name}")

scalar variable: A(1)f
scalar variable with affined transformation: A(1,[Af(2,-1)])f


`nevergrad` uses specific terms to denote the _user_ - level variables ("arguments") and _internal_ variables ("data"). 

An example of how they are related:

In [46]:
print("'data' to 'arguments' for `scalar_variable`: ", scalar_variable.data_to_arguments([4.0]))
print("'arguments' to 'data' for `scalar_variable`: ", scalar_variable.arguments_to_data([4.0]))

'data' to 'arguments' for `scalar_variable`:  ((4.0,), {})
'arguments' to 'data' for `scalar_variable`:  [[4.]]


In [47]:
print("'data' to 'arguments' for `affined_scalar_variable`: ", affined_scalar_variable.data_to_arguments([4.0]))
print("'arguments' to 'data' for `affined_scalar_variable`: ", affined_scalar_variable.arguments_to_data([4.0]))

'data' to 'arguments' for `affined_scalar_variable`:  ((7.0,), {})
'arguments' to 'data' for `affined_scalar_variable`:  [[2.5]]


## Using `Array` variables

In fact, `Scalar` is a special one dimensional case of an `Array` variable:

In [57]:
array_variable = ng.var.Array(2, 2)
print(f"Array variable: {array_variable.name}")
print(f"Array dimension: {array_variable.dimension}")

user_argument = [
    [-2.0, 2.0], 
    [-2.0, 2.0]
]
print(f"\n'arguments' to 'data' for `array_variable`: {array_variable.arguments_to_data(user_argument)}")
print(f"\n'data' to 'arguments' for `affined_scalar_variable`: \n{array_variable.data_to_arguments([-2, 2, -2, 2])}")

Array variable: A(2,2)
Array dimension: 4

'arguments' to 'data' for `array_variable`: [-2.  2. -2.  2.]

'data' to 'arguments' for `affined_scalar_variable`: 
((array([[-2,  2],
       [-2,  2]]),), {})


## Using transformations

`affined` is one of the transformations we can apply to our variables.

Let's try something new:

In [75]:
affined_variable = ng.var.Array(1, 2).affined(2, 10)
print(f"affined_variable: {affined_variable.name}")
print(f"`affined_variable` data to arguments: {affined_variable.data_to_arguments([-2.0, 10.0])}\n\n")

bounded_affined_variable = ng.var.Array(1, 2).affined(2, 10).bounded(7., 20., transform="clipping")
print(f"bounded_affined_variable: {bounded_affined_variable.name}")
print(f"`bounded_affined_variable` data to arguments: {bounded_affined_variable.data_to_arguments([-2.0, 10.0])}")

affined_variable: A(1,2,[Af(2,10)])
`affined_variable` data to arguments: ((array([[ 6., 30.]]),), {})


bounded_affined_variable: A(1,2,[Af(2,10),Cl(7.0,20.0)])
`bounded_affined_variable` data to arguments: ((array([[ 7., 20.]]),), {})


You can stack `transformations` in any order you like.

## Using discrete variables

Dealing with discrete variables was one of the most useful features of `nevergrad` for the Force-BDSS optimizer.

### `OrderedDiscrete`

From the official documentation:

```
OrderedDiscrete(items): converts a list of (ordered) discrete items into a 1-dimensional variable. 
The returned value will depend on the value on this dimension: low values corresponding to first elements of the list, and high values to the last.
```

In [124]:
ordered_data = ["a", "b", "c", "d"]
ordered_discrete = ng.var.OrderedDiscrete(ordered_data)

Conversion from the internal data space to user arguments:

In [126]:
print(f"ordered_discrete.dimension = {ordered_discrete.dimension}")
print(f"`data = -0.68` to arguments: {ordered_discrete.data_to_arguments([-0.68])}")
print(f"`data = -0.67` to arguments: {ordered_discrete.data_to_arguments([-0.67])}")
print(f"`data =  0.67` to arguments: {ordered_discrete.data_to_arguments([0.67])}")
print(f"`data =  0.68` to arguments: {ordered_discrete.data_to_arguments([0.68])}")

ordered_discrete.dimension = 1
`data = -0.68` to arguments: (('a',), {})
`data = -0.67` to arguments: (('b',), {})
`data =  0.67` to arguments: (('c',), {})
`data =  0.68` to arguments: (('d',), {})


Converting from the user arguments space to the internal data:

In [125]:
for parameter in ordered_data:
    print(f"(argument = {parameter}) to data: {ordered_discrete.arguments_to_data(parameter)}")

(argument = a) to data: [-1.15034938]
(argument = b) to data: [-0.31863936]
(argument = c) to data: [0.31863936]
(argument = d) to data: [1.15034938]


### `SoftmaxCategorical`

From the official documentation:
```
SoftmaxCategorical(items): converts a list of n (unordered) categorial items into an n-dimensional space. 
The returned element will be sampled as the softmax of the values on these dimensions. 
Be cautious: this process is non-deterministic and makes the function evaluation noisy.
```

In [129]:
softmax_data = ["x", "y", "z"]

softmax_discrete = ng.var.SoftmaxCategorical(softmax_data)
print(f"softmax_discrete.dimension = {softmax_discrete.dimension}")

softmax_discrete.dimension = 3


Conversion from the internal data space to user arguments:

In [128]:
print(f"`data = [100, 1, 1]` to arguments: {softmax_discrete.data_to_arguments([100, 1, 1])}")
print(f"`data = [1, 100, 1]` to arguments: {softmax_discrete.data_to_arguments([1, 100, 1])}")
print(f"`data = [1, 1, 100]` to arguments: {softmax_discrete.data_to_arguments([1, 1, 100])}")

`data = [100, 1, 1]` to arguments: (('x',), {})
`data = [1, 100, 1]` to arguments: (('y',), {})
`data = [1, 1, 100]` to arguments: (('z',), {})


Converting from the user arguments space to the internal data:

In [130]:
for parameter in softmax_data:
    print(f"(argument = {parameter}) to data: {softmax_discrete.arguments_to_data(parameter)}")

(argument = x) to data: [0.69314718 0.         0.        ]
(argument = y) to data: [0.         0.69314718 0.        ]
(argument = z) to data: [0.         0.         0.69314718]


# Assembling `Instrumentation`

From the official documentation:

In [132]:
arg1 = ng.var.OrderedDiscrete(["a", "b"])  # 1st arg. = positional discrete argument
arg2 = ng.var.SoftmaxCategorical(["a", "c", "e"])  # 2nd arg. = positional discrete argument
value = ng.var.Gaussian(mean=1, std=2)  # the 4th arg. is a keyword argument with Gaussian prior

# create the instrumented function
instrum = ng.Instrumentation(arg1, arg2, "blublu", value=value)
# the 3rd arg. is a positional arg. which will be kept constant to "blublu"
print("Instrumentation dimension: ", instrum.dimension)  # 5 dimensional space


print(instrum.data_to_arguments([1, -80, -80, 80, 3]))
# prints (args, kwargs): (('b', 'e', 'blublu'), {'value': 7})
# b is selected because 1 > 0 (the threshold is 0 here since there are 2 values.
# e is selected because proba(e) = exp(80) / (exp(80) + exp(-80) + exp(-80))
# value=7 because 3 * std + mean = 7

Instrumentation dimension:  5
(('b', 'e', 'blublu'), {'value': 7})


In [138]:
def myfunction(arg1, arg2, arg3, value=3):
    # print(arg1, arg2, arg3)
    return value**2

optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=100)
recommendation = optimizer.minimize(myfunction)
print(recommendation)

Candidate(args=('b', 'a', 'blublu'), kwargs={'value': 1.4090111434383878e-05})


# "Ask and Tell"

From the official documentation:

An ask and tell interface is also available. The 3 key methods for this interface are respectively:

`ask`: suggest a candidate on which to evaluate the function to optimize.

`tell`: for updated the optimizer with the value of the function for a candidate.

`provide_recommendation`: returns the candidate the algorithms considers the best.

In [144]:
def square(x, y=12):
    """
    Convex objective function
    """
    return sum((x - 1.5) ** 2) + abs(y)

instrum = ng.Instrumentation(ng.var.Array(2), y=ng.var.Array(1).asscalar())
optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=100)

for _ in range(optimizer.budget):
    x = optimizer.ask()
    value = square(*x.args, **x.kwargs)
    optimizer.tell(x, value)


recommendation = optimizer.provide_recommendation()
print(recommendation)

Candidate(args=(array([1.48997154, 1.41973771]),), kwargs={'y': -0.004140369520403001})
